In [1]:
import tensorflow as tf
import os

In [2]:
import cv2
import imghdr

In [3]:
# data_dir = 'data' 
data_dir = 'combined copy' 

In [4]:
import numpy as np
from matplotlib import pyplot as plt

In [5]:
data = tf.keras.utils.image_dataset_from_directory('combined copy')

Found 5793 files belonging to 2 classes.


In [6]:
data_iterator = data.as_numpy_iterator()

In [7]:
batch = data_iterator.next()

2023-03-29 23:51:01.909125: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


In [9]:
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx, img in enumerate(batch[0][:4]):
    ax[idx].imshow(img.astype(int))
    ax[idx].title.set_text(batch[1][idx])

In [8]:
data = data.map(lambda x,y: (x/255, y))

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [9]:
train_size = int(len(data)*.7)
val_size = int(len(data)*.2)
test_size = int(len(data)*.1)

In [10]:
train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size+val_size).take(test_size)

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout

In [ ]:
# model = tf.keras.models.Sequential()
# model.add(Conv2D(16, (3,3), 1, activation='relu', input_shape=(256,256,3)))
# model.add(MaxPooling2D())
# model.add(Conv2D(32, (3,3), 1, activation='relu'))
# model.add(MaxPooling2D())
# model.add(Conv2D(16, (3,3), 1, activation='relu'))
# model.add(MaxPooling2D())
# model.add(Flatten())
# model.add(Dense(256, activation='relu'))
# model.add(Dense(1, activation='sigmoid'))

In [12]:
model2 = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3, 3), activation="relu", input_shape=(256, 256, 3), kernel_initializer='he_uniform'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(32, (3, 3), activation="relu",kernel_initializer='he_uniform'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(16, (3, 3), activation="relu",kernel_initializer='he_uniform'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation="relu",kernel_initializer='he_uniform'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation="sigmoid",kernel_initializer='glorot_uniform')
])

In [13]:
model2.compile('adam', loss=tf.losses.BinaryCrossentropy(), metrics=['accuracy'])

In [14]:
model2.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 127, 127, 16)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 32)      4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 62, 62, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 60, 60, 16)        4624      
                                                                 
 flatten (Flatten)           (None, 57600)             0

In [15]:
logdir='logs'
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [16]:
hist = model2.fit(train, epochs=20, validation_data=val, callbacks=[tensorboard_callback])

Epoch 1/20
127/127 [==============================] - 52s 400ms/step - loss: 2.3142 - accuracy: 0.5544 - val_loss: 0.6790 - val_accuracy: 0.6102
Epoch 2/20
127/127 [==============================] - 55s 427ms/step - loss: 0.5728 - accuracy: 0.7170 - val_loss: 0.6124 - val_accuracy: 0.6649
Epoch 3/20
127/127 [==============================] - 58s 456ms/step - loss: 0.4134 - accuracy: 0.8250 - val_loss: 0.5846 - val_accuracy: 0.6866
Epoch 4/20
127/127 [==============================] - 57s 445ms/step - loss: 0.2412 - accuracy: 0.9090 - val_loss: 0.5938 - val_accuracy: 0.7109
Epoch 5/20
127/127 [==============================] - 59s 463ms/step - loss: 0.1369 - accuracy: 0.9545 - val_loss: 0.6323 - val_accuracy: 0.7474
Epoch 6/20
127/127 [==============================] - 60s 472ms/step - loss: 0.0687 - accuracy: 0.9801 - val_loss: 0.8356 - val_accuracy: 0.7274
Epoch 7/20
127/127 [==============================] - 65s 507ms/step - loss: 0.0361 - accuracy: 0.9904 - val_loss: 0.7602 - val_ac

In [17]:
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy, 
pre = Precision()
re = Recall()
acc = BinaryAccuracy()
f=F
for batch in test.as_numpy_iterator(): 
    X, y = batch
    yhat = model2.predict(X)
    pre.update_state(y, yhat)
    re.update_state(y, yhat)
    acc.update_state(y, yhat)
print(pre.result(), re.result(), acc.result())

1/1 [==============================] - 0s 90ms/step
tf.Tensor(0.6692913, shape=(), dtype=float32) tf.Tensor(0.609319, shape=(), dtype=float32) tf.Tensor(0.6649306, shape=(), dtype=float32)


In [6]:
from tensorflow.keras.models import load_model
model2.save(os.path.join('models','softstoryclassifier.h5'))

NameError: name 'model2' is not defined

In [20]:
model_json = model2.to_json()
with open("model4.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5


In [5]:
#from keras import model_from_json 

# opening and store file in a variable

json_file = open('model4.json','r')
loaded_model_json = json_file.read()
json_file.close()

# use Keras model_from_json to make a loaded model

loaded_model = tf.keras.models.model_from_json(loaded_model_json)

# load weights into new model

loaded_model.load_weights("softstoryclassifier.h5")
print("Loaded Model from disk")

# compile and evaluate loaded model

loaded_model.compile(loss=tf.losses.BinaryCrossentropy(),optimizer='adam',metrics=['accuracy'])


NameError: name 'tf' is not defined